In [3]:
def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

# Example usage
input_string = 'AGATAAGG'
sorted_substrings = generate_sorted_substrings(input_string)

sorted_substrings

['AGATAAGG',
 'AGATAAG',
 'GATAAGG',
 'AGATAA',
 'ATAAGG',
 'GATAAG',
 'AGATA',
 'ATAAG',
 'GATAA',
 'TAAGG',
 'AAGG',
 'AGAT',
 'ATAA',
 'GATA',
 'TAAG',
 'AAG',
 'AGA',
 'AGG',
 'ATA',
 'GAT',
 'TAA',
 'AA',
 'AG',
 'AT',
 'GA',
 'GG',
 'TA',
 'A',
 'G',
 'T']

In [ ]:
import pandas as pd

def read_sequences_from_file(file_path):
    with open(file_path, 'r') as file:
        next(file)
        sequences = []
        for line in file:
            columns = line.split()
            sequences.append(columns[0].replace('.',''))
    return sequences

def ref_seq(sequences):
    for sequence in sequences:
        return sequence
    return None

def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

def align_sequences(sequences, reference_sequence):
    ref_index = 0
    substrings = generate_sorted_substrings(reference_sequence)

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        if reference_sequence in sequence:
            idx = sequence.find(reference_sequence)
            aligned = True
            entire = True
        else:
            for substring in substrings:
                if substring in sequence:
                    idx = sequence.find(substring)
                    ref_index2 = reference_sequence.find(substring)
                    entire = False
                    aligned = True
                    break

        if aligned:
            if entire:
                offset = ref_index - idx
            else:
                offset = ref_index2 - idx
            indices = range(offset, offset + len(sequence))
            df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
        
    return df

for i in ["Cebpa","Cebpb"]:
    input_file_path = f'/Users/bristi/Desktop/Design Project/Working-with-TF/GATA4/GATA4_ SMALL_testing.txt'
    output_file_path = 'new_algo_GATA_aligned_sequences.txt'

    sequences = read_sequences_from_file(input_file_path)
    
    reference_sequence = sequences[0]
    if not reference_sequence:
        raise ValueError(f"No sequence containing '{reference_sequence}' was found in the file.")

    df_aligned = align_sequences(sequences, reference_sequence)
    
    columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
    extracted_data = df_aligned[columns_to_extract]
    extracted_data = extracted_data.fillna('-')

    formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

    with open(output_file_path, 'w') as file:
        for line in formatted_data:
            file.write(line + '\n')

In [9]:
df_aligned

,-7,-6,-5,-4,-3,-2,-1,0,1,2,...,7,8,9,10,11,12,13,14,15,16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,...,A,G,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,A,C,T,T,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,...,A,G,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,G,...,A,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,C,...,A,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,T,C,C,C,C,C,T,A
58062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,T,A,NaN,NaN,NaN,NaN,NaN,NaN
58063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,A,T,C,T,A,T,C,C,NaN
58064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN


# Generating txt file for weblogo

In [ ]:
columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
extracted_data = df_aligned[columns_to_extract]
extracted_data = extracted_data.fillna('-')

formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

output_file_path = 'Elf2_whole_aligned.txt'
with open(output_file_path, 'w') as file:
    for line in formatted_data:
        file.write(line + '\n')

# Better Algo

In [37]:
import pandas as pd

def read_sequences_from_file(file_path):
    with open(file_path, 'r') as file:
        next(file)
        sequences = []
        for line in file:
            columns = line.split()
            sequences.append(columns[0].replace('.',''))
    return sequences

def ref_seq(sequences):
    for sequence in sequences:
        return sequence
    return None

def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

def align_sequences(sequences, seed):
    ref_index = 0
    substrings = generate_sorted_substrings(seed)

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        if seed in sequence:
            idx = sequence.find(seed)
            aligned = True
            entire = True
        else:
            prev_best_number_of_matches = 0
            total_number_of_matches = 0
            best_substring_index = -1
        
            for i in range(0, len(substrings)):
                if substrings[i] in sequence:   
                    for j in range(i-1, len(substrings)):
                        if substrings[j] in sequence:
                            total_number_of_matches += 1
                    
                    if prev_best_number_of_matches < total_number_of_matches:
                        best_substring_index = i
                    
            
            best_substring = substrings[best_substring_index]
            idx = sequence.find(best_substring)
            ref_index2 = seed.find(best_substring)
            entire = False
            aligned = True
                    

        if aligned:
            if entire:
                offset = ref_index - idx
            else:
                offset = ref_index2 - idx
            indices = range(offset, offset + len(sequence))
            df = pd.concat([df, pd.DataFrame([list(sequence)], columns=indices)], ignore_index=True)

        
    return df
#"Cebpa","Cebpb"
for i in ['GATA']:
    input_file_path = f'/Users/bristi/Desktop/Design Project/Working-with-TF/TFs UniProbe Data/GATA4/GATA4_ SMALL_testing.txt'
    output_file_path = f'prev_new_algo_whole_{i}_aligned_sequences.txt'

    sequences = read_sequences_from_file(input_file_path)

    seed = sequences[0]
    if not seed:
        raise ValueError(f"No sequence containing '{seed}' was found in the file.")

    df_aligned = align_sequences(sequences, seed)

    columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
    extracted_data = df_aligned[columns_to_extract]
    extracted_data = extracted_data.fillna('-')

    formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

    with open(output_file_path, 'w') as file:
        for line in formatted_data:
            file.write(line + '\n')

# Correct variation

In [4]:
import pandas as pd

def read_sequences_from_file(file_path):
    with open(file_path, 'r') as file:
        next(file)
        sequences = []
        for line in file:
            columns = line.split()
            sequences.append(columns[0].replace('.',''))
    return sequences

def ref_seq(sequences):
    for sequence in sequences:
        return sequence
    return None

def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

def align_sequences(sequences, reference_sequence):
    ref_index = 0
    substrings = generate_sorted_substrings(reference_sequence)

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        if reference_sequence in sequence:
            idx = sequence.find(reference_sequence)
            aligned = True
            entire = True
        else:
            prev_best_number_of_matches = 0
            total_number_of_matches = 0
            best_substring_index = -1
                    
            for i in range(0, len(substrings)):
                if substrings[i] in sequence:                    
                    currect_substring = substrings[i]
                    idx = sequence.find(currect_substring)
                    substring_index_in_seed = reference_sequence.find(currect_substring)
                    offset = substring_index_in_seed - idx
                    
                    if offset < 0:
                        aligned_seed = reference_sequence[:len(reference_sequence) - abs(offset)]
                        aligned_sequence = sequence[abs(offset):]
                        
                    else:
                        aligned_seed = reference_sequence[abs(offset):]
                        aligned_sequence = sequence[:len(sequence) - abs(offset)]
                    
                    aligned_seed_substrings = generate_sorted_substrings(aligned_seed)
                    for substring in aligned_seed_substrings:
                        if substring in aligned_sequence:
                            total_number_of_matches += 1
                    
                    if prev_best_number_of_matches < total_number_of_matches:
                        best_substring_index = i
                        
            
            best_substring = substrings[best_substring_index]
            idx = sequence.find(best_substring)
            substring_index_in_seed = reference_sequence.find(best_substring)
            entire = False
            aligned = True       


        if aligned:
            if entire:
                offset = ref_index - idx
            else:
                offset = substring_index_in_seed - idx
            indices = range(offset, offset + len(sequence))
            df = pd.concat([df, pd.DataFrame([list(sequence)], columns=indices)], ignore_index=True)

        
    return df
#"Cebpa","Cebpb"
for i in ['Cebpa','Cebpb']:
    input_file_path = f'/Users/bristi/Desktop/Design Project/Working-with-TF/TFs UniProbe Data/{i}/{i}_8mers_top_enrichment.txt'
    output_file_path = f'correct_new_algo_whole_{i}_aligned_sequences.txt'

    sequences = read_sequences_from_file(input_file_path)

    reference_sequence = sequences[0]
    if not reference_sequence:
        raise ValueError(f"No sequence containing '{reference_sequence}' was found in the file.")

    df_aligned = align_sequences(sequences, reference_sequence)

    columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
    extracted_data = df_aligned[columns_to_extract]
    extracted_data = extracted_data.fillna('-')

    formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

    with open(output_file_path, 'w') as file:
        for line in formatted_data:
            file.write(line + '\n')

# Length to consider after alignment

In [20]:
currect_substring = 'AT'

reference_sequence = 'AGATAAGG'
sequence = 'CTTATCGC'

idx = sequence.find(currect_substring)
substring_index_in_seed = reference_sequence.find(currect_substring)
offset = substring_index_in_seed - idx

if offset < 0:
    print(f"Consider seed from 0 to {len(reference_sequence) - abs(offset)}")
    print(f"Consider sequence from {abs(offset)} to {len(sequence)}")
    print(reference_sequence[0:len(reference_sequence) - abs(offset)])
    print(sequence[abs(offset):])
else:
    print(f"Consider seed from {abs(offset)} to {len(reference_sequence)}")
    print(f"Consider sequence from 0 to {len(sequence) - abs(offset)}")
    print(reference_sequence[abs(offset):])
    print(sequence[:len(sequence) - abs(offset)])
    

Consider seed from 0 to 3
Consider sequence from 5 to 8
AGA
CGC


# WebLogo 3 access

In [6]:
from weblogo import *

fin = open('/Users/bristi/Desktop/Design Project/Working-with-TF/Aligning Sequences/correct_new_algo_whole_Cebpb_aligned_sequences.txt')
seqs = read_seq_data(fin)
logodata = LogoData.from_seqs(seqs)
logooptions = LogoOptions()
logooptions.title = "A Logo Title"
logooptions.color_scheme = std_color_schemes['classic']
logoformat = LogoFormat(logodata, logooptions)
png_bytes = png_print_formatter(logodata, logoformat)

with open('cebpb.png', 'wb') as fout:
    fout.write(png_bytes)

# Parallelisation Attempt

In [3]:
import pandas as pd
import multiprocessing
from multiprocessing import Pool

def read_sequences_from_file(file_path):
    with open(file_path, 'r') as file:
        next(file)
        sequences = []
        for line in file:
            columns = line.split()
            sequences.append(columns[0].replace('.', ''))
    return sequences

def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]

    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))

    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)

    return unique_substrings

def align_single_sequence(args):
    sequence, reference_sequence, substrings, ref_index = args
    aligned = False
    entire = False
    df_row = pd.DataFrame()

    if reference_sequence in sequence:
        idx = sequence.find(reference_sequence)
        aligned = True
        entire = True
    else:
        prev_best_number_of_matches = 0
        total_number_of_matches = 0
        best_substring_index = -1

        for i in range(len(substrings)):
            if substrings[i] in sequence:
                current_substring = substrings[i]
                idx = sequence.find(current_substring)
                substring_index_in_seed = reference_sequence.find(current_substring)
                offset = substring_index_in_seed - idx

                if offset < 0:
                    aligned_seed = reference_sequence[:len(reference_sequence) - abs(offset)]
                    aligned_sequence = sequence[abs(offset):]
                else:
                    aligned_seed = reference_sequence[abs(offset):]
                    aligned_sequence = sequence[:len(sequence) - abs(offset)]

                aligned_seed_substrings = generate_sorted_substrings(aligned_seed)
                for substring in aligned_seed_substrings:
                    if substring in aligned_sequence:
                        total_number_of_matches += 1

                if prev_best_number_of_matches < total_number_of_matches:
                    prev_best_number_of_matches = total_number_of_matches
                    best_substring_index = i

        if best_substring_index != -1:
            best_substring = substrings[best_substring_index]
            idx = sequence.find(best_substring)
            substring_index_in_seed = reference_sequence.find(best_substring)
            entire = False
            aligned = True

    if aligned:
        if entire:
            offset = ref_index - idx
        else:
            offset = substring_index_in_seed - idx
        indices = range(offset, offset + len(sequence))
        df_row = pd.DataFrame([list(sequence)], columns=indices)

    return df_row

def align_sequences(sequences, reference_sequence):
    ref_index = 0
    substrings = generate_sorted_substrings(reference_sequence)
    columns = [i for i in range(-7, 15)]
    df = pd.DataFrame(columns=columns)

    # Prepare arguments for parallel processing
    args_list = [(sequence, reference_sequence, substrings, ref_index) for sequence in sequences]

    # Use multiprocessing Pool to process sequences in parallel
    with Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(align_single_sequence, args_list)

    # Combine results into a single DataFrame
    for df_row in results:
        if not df_row.empty:
            df = pd.concat([df, df_row], ignore_index=True)

    return df

if __name__ == '__main__':
    # "Cebpa","Cebpb"
    for i in ['Cebpa']:
        input_file_path = f'/Users/bristi/Desktop/Design Project/Working-with-TF/TFs UniProbe Data/GATA4/GATA4_ SMALL_testing.txt'
        output_file_path = f'parallel_{i}_aligned_sequences.txt'

        sequences = read_sequences_from_file(input_file_path)

        reference_sequence = sequences[0]
        if not reference_sequence:
            raise ValueError(f"No sequence containing '{reference_sequence}' was found in the file.")

        df_aligned = align_sequences(sequences, reference_sequence)

        columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
        extracted_data = df_aligned[columns_to_extract]
        extracted_data = extracted_data.fillna('-')

        formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

        with open(output_file_path, 'w') as file:
            for line in formatted_data:
                file.write(line + '\n')

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/bristi/miniforge3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/bristi/miniforge3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bristi/miniforge3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/bristi/miniforge3/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'align_single_sequence' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/bristi/miniforge3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/bristi/miniforge3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bristi/miniforge3/l

KeyboardInterrupt: 